In [4]:
import os
import whisper
import subprocess

os.environ["PATH"] += os.pathsep + r"C:\ffmpeg\bin"

def check_ffmpeg():
    try:
        cmd = ["ffmpeg", "-version"]
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0:
            print("ffmpeg is accessible")
        else:
            print("ffmpeg is not accessible")
    except FileNotFoundError:
        print("ffmpeg not found")

check_ffmpeg()

model = whisper.load_model("base")

def transcribe_audio(file_path):
    audio = whisper.load_audio(file_path)
    audio = whisper.pad_or_trim(audio)
    
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")
    
    options = whisper.DecodingOptions(fp16=False)
    result = whisper.decode(model, mel, options)
    
    print(f"Transcription: {result.text}")
    return result.text

flac_file_path = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac'
print(flac_file_path)

if os.path.isfile(flac_file_path):
    transcription = transcribe_audio(flac_file_path)
    print(f"Final Transcription: {transcription}")
else:
    print("File not found. Please check the file path.")


ffmpeg not found
C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
# for multiple processing
import whisper
import os

# Load the Whisper model
model = whisper.load_model("base")

# Function to transcribe audio file
def transcribe_audio(file_path):
    audio = whisper.load_audio(file_path)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")
    options = whisper.DecodingOptions(fp16=False)
    result = whisper.decode(model, mel, options)
    print(f"Transcription: {result.text}")
    return result.text

# Directory containing WAV files
wav_directory = "path/to/your/wav_files"

# Transcribe all WAV files in the directory
transcriptions = {}
for filename in os.listdir(wav_directory):
    if filename.endswith(".wav"):
        file_path = os.path.join(wav_directory, filename)
        transcription = transcribe_audio(file_path)
        transcriptions[filename] = transcription

# Print all transcriptions
for filename, transcription in transcriptions.items():
    print(f"{filename}: {transcription}")


In [20]:
# save transcribe
import whisper
import os

# Load the Whisper model
model = whisper.load_model("base")

# Function to transcribe audio file
def transcribe_audio(file_path):
    audio = whisper.load_audio(file_path)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")
    options = whisper.DecodingOptions(fp16=False)
    result = whisper.decode(model, mel, options)
    print(f"Transcription: {result.text}")
    return result.text

# Directory containing WAV files
wav_directory = "path/to/your/wav_files"

# Transcribe all WAV files in the directory and save to a text file
with open("transcriptions.txt", "w") as f:
    for filename in os.listdir(wav_directory):
        if filename.endswith(".wav"):
            file_path = os.path.join(wav_directory, filename)
            transcription = transcribe_audio(file_path)
            f.write(f"{filename}: {transcription}\n")


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'path/to/your/wav_files'

In [ ]:
# Objective Evaluation
# Word Error Rate (WER):

# Transcribe the generated audio back to text using an automatic speech recognition (ASR) system.
# Compare the transcribed text with the original text to calculate the WER. The model with the lower WER is better.

In [7]:
from jiwer import wer

original_text = "The original text from the dataset."
transcribed_text_model1 = "The original text from the database."
transcribed_text_model2 = "The transcribed text from model 2."

wer_model1 = wer(original_text, transcribed_text_model1)
wer_model2 = wer(original_text, transcribed_text_model2)

print(f"WER Model 1: {wer_model1}")
print(f"WER Model 2: {wer_model2}")


WER Model 1: 0.16666666666666666
WER Model 2: 0.5


In [ ]:
# Perceptual Evaluation of Speech Quality (PESQ):

# Use PESQ to measure the quality of the speech signals. PESQ compares the generated audio to a reference audio signal and gives a quality score.
# Libraries like pypesq can be used for this purpose.

In [10]:
from pypesq import pesq
from scipy.io import wavfile

rate, ref_audio = wavfile.read(r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac.wav')
rate, gen_audio_model1 = wavfile.read(r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac.wav')
rate, gen_audio_model2 = wavfile.read(r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00001.flac.wav')

pesq_score_model1 = pesq(rate, ref_audio, gen_audio_model1, 'wb')
pesq_score_model2 = pesq(rate, ref_audio, gen_audio_model2, 'wb')

print(f"PESQ Score Model 1: {pesq_score_model1}")
print(f"PESQ Score Model 2: {pesq_score_model2}")


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()